In [ ]:
subway = pl.read_csv('subway_feature.csv')
subway = subway.rename({'경도':'X','위도':'Y'})
subway = subway.group_by(['역사명', '호선']).agg(
    pl.col('X').mean(),
    pl.col('Y').mean()
)

# degrees to radians
main_coords = np.radians(main.select(['lat', 'long']).to_numpy())
subway_coords = np.radians(subway.select(['Y', 'X']).to_numpy())  
test_coords = np.radians(test.select(['lat', 'long']).to_numpy())

# haversine formula
def haversine(a, b):
    dlat = b[:, None, 0] - a[None, :, 0]
    dlon = b[:, None, 1] - a[None, :, 1]
    r = 6371  

    hav = (
        np.sin(dlat / 2)**2
        + np.cos(a[None, :, 0]) * np.cos(b[:, None, 0]) * np.sin(dlon / 2)**2
    )
    return 2 * r * np.arcsin(np.sqrt(hav)).T  

sub_dists = haversine(main_coords, subway_coords) 
test_dists = haversine(test_coords, subway_coords) 

condition = sub_dists <= 1
counts = condition.sum(axis=1)
condition2 = test_dists <= 1
test_counts = condition2.sum(axis=1)
main = main.with_columns(
    pl.Series("nearby_subway_count", counts)
)
test = test.with_columns(
    pl.Series("nearby_subway_count", test_counts)
)